In [1]:
import langchain

print(langchain.__version__)

1.1.0


课程环境安装的Langchain版本为1.1.0，在开始之前，请先安装，为了保证一致性，尽量使用相同的版本：

```bash
pip install langchain==1.0.0
```

模型是智能体的推理引擎。它们驱动着智能体的决策过程，决定调用哪些工具、如何解释结果以及何时提供最终答案。

LangChain本身不提供模型，而是为我们封装了一套与主流大语言模型进行交互的标准构建模块，以DeepSeek系列模型为例，如果我们想通过 LangChain接入DeepSeek的模型，就需要在LangChain框架下通过预定义好的类和方法来规定如何与模型进行交互，包括数据的输入和输出格式以及如何连接到模型本身。国内外主流模型都有预开发好的专属的类，绝大多数模型都遵循OpenAI的规范，没有专属类也不需要担心，直接用OpenAI的类也能实例化能够在LangChain中运行的实例，对于有些公司内部部署的模型，在OpenAI类上进行二开即可。

通过这个链接查看所有的有预开发好的包：https://docs.langchain.com/oss/python/integrations/providers/all_providers

# 设置环境变量

开发环境中，为了方便我们直接把相关的API Key保存到.env文件中：  
DEEPSEEK_API_KEY=sk-xxx  
DASHSCOPE_API_KEY=sk-xxx  
DASHSCOPE_BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1

In [ ]:
from dotenv import load_dotenv
import os

from langchain_community.embeddings import dashscope

load_dotenv()  # 自动查找当前目录的 .env 文件

dashscope_api_key = os.getenv("DASHSCOPE_API_KEY")
dashscope_base_url = os.getenv("DASHSCOPE_BASE_URL")

deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")


生产环境，一般设置在系统环境变量中保证安全不泄露。

# init_chat_model

在 LangChain 中初始化模型最简单的方法是使用 init_chat_model 从聊天模型提供商处初始化一个模型，init_chat_model接收以下参数：

model参数是必填项，属于字符串类型，由LangChain来推断，国内模型仅支持deepseek的，这里我们用deepseek来演示。

🔴 [查看支持的模型提供商](https://reference.langchain.com/python/langchain/models/?_gl=1*1dk8e7h*_gcl_au*MTEwMDAzMDA1Mi4xNzY3OTYxMTMx*_ga*NDA5ODUyNzYwLjE3Njc5NjExMzI.*_ga_47WX3HKKY2*czE3NjkzMjE5MDEkbzEwJGcxJHQxNzY5MzI0NDc2JGoxNiRsMCRoMA..#langchain.chat_models.init_chat_model(model))


In [13]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "deepseek-chat",
    # 可以省略
    api_key=deepseek_api_key,
    temperature=0.7,
)

In [12]:
result = model.invoke(
    "介绍一下你自己"
)

print(result)

content='你好！我是DeepSeek，一个由深度求索公司创造的AI助手，很高兴认识你！😊\n\n让我简单介绍一下自己：\n\n**我的特点：**\n- 🆓 **完全免费**：目前没有任何收费计划，你可以放心使用\n- 📚 **知识丰富**：知识截止到2024年7月，涵盖各个领域\n- 💬 **对话流畅**：支持128K上下文，能记住我们长时间的对话内容\n- 📁 **文件处理**：支持上传图像、txt、pdf、ppt、word、excel等多种格式文件，并能从中读取文字信息进行处理\n- 🌐 **联网搜索**：支持联网功能（需要你在Web/App中手动开启）\n- 📱 **多平台使用**：可通过官方应用商店下载App使用\n\n**我能帮你做什么：**\n- 回答各种问题，提供信息和解释\n- 协助写作、翻译、编程等任务\n- 分析和处理你上传的文档\n- 进行创意思考和问题解决\n- 日常聊天和交流\n\n**我的风格：**\n我会以热情、细腻的方式与你交流，尽力提供准确、有用的帮助。虽然我是纯文本模型，但我会用丰富的文字描述来弥补这一点！\n\n有什么想了解的或需要帮助的吗？我很乐意为你服务！✨' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 267, 'prompt_tokens': 6, 'total_tokens': 273, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 6}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '80f08e78-4781-42ff-8aba-3023042feeb1', 'finish_reason':

## 常见的参数

常见的**kwargs参数：
- temperature：用于控制模型输出随机性的温度参数。
- max_tokens：生成结果的最大 token 数量。
- timeout：等待模型响应的最长时间（以秒为单位）。
- max_retries：请求失败时的最大重试次数。
- base_url：自定义的 API 端点 URL。
- rate_limiter：一个 BaseRateLimiter 实例，用于控制请求频率。

## 特殊参数

### configurable_fields

用来控制在运行过程中可以传入配置项的参数，<span style="color:red">实现动态调整参数</span>。运行时可传入的参数我们放到Lesson2中介绍，这里下简单举个例子。

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "deepseek-chat",
    # 可以省略
    api_key=deepseek_api_key,
    temperature=0.7,
    # 运行时动态指定的max_tokens参数生效
    configurable_fields=["max_tokens"],
)

In [ ]:
result = model.invoke(
    "介绍一下你自己",
    config = {
        "configurable": {
        # 配置项生效，最终输出最多为32个token
            "max_tokens": "32"
        }
    }
)

print(result.response_metadata["token_usage"]["completion_tokens"])


### config_prefix

指定configurable参数的前缀

In [23]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "deepseek-chat",
    # 可以省略
    api_key=deepseek_api_key,
    temperature=0.7,
    # 运行时动态指定的max_tokens参数生效
    configurable_fields=["max_tokens"],
    config_prefix="foo",
)

In [ ]:
result = model.invoke(
    "介绍一下你自己",
    config = {
        "configurable": {
        # 配置项必须加上前缀才生效
            "foo_max_tokens": "32"
        }
    }
)

print(result.response_metadata["token_usage"]["completion_tokens"])

# 专属类初始化模型

国内外热门模型都要官方提供的专属类来初始化，这里以Deepseek模型为例，展示如何用专属类初始化模型，首先安装相应的依赖：

```bash
pip install -U "langchain[deepseek]
```

In [33]:
# key配置到环境变量DEEPSEEK_API_KEY，否则要显性传入

from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(model="deepseek-chat", temperature=0.7)


In [ ]:
result = model.invoke(
    "介绍一下你自己",
)

print(result.content)

# ChatOpenAI泛用类初始化模型

这里以千问的模型为例，演示非专属类初始化模型。首先安装相应的依赖：

```bash
pip install -U "langchain[openai]"
```

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="qwen-plus",
    api_key=dashscope_api_key,
    base_url=dashscope_base_url
)

response = model.invoke("介绍一下你自己")
print(response)

但是这样做容易出现兼容性问题，比如上述案例，经过测试发现max_tokens参数并不生效，其余的大家可以自行测试，因此优先使用专属类或者init_chat_model方法来初始化模型。

🔴 [查看所有的专属Chat类](https://docs.langchain.com/oss/python/integrations/chat)